In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
import ternary
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

# ---- Funções auxiliares do diagrama ----

def rot_list(lst):
    return [[p[0], p[1], p[2]] for p in lst]

def linha_const(c):
    pts = []
    for s in np.linspace(0, 100 - c, 200):
        si = 100 - s - c
        pts.append([s, c, si])
    return pts

lista1 = [[33,0,67],[3.3,90.0,6.7]]
lista2 = [[67,0,33],[6.7,90.0,3.3]]

# Nome das regiões
regiao_areias        = [4,92,4]
regiao_siltes        = [80,2,18]
regiao_argilas       = [15,2,83]
regiao_lama          = [50,2,48]
regiao_argilaArenosa = [15,12,73]
regiao_lamaArenosa   = [45,12,43]
regiao_silteArenoso  = [73,12,10]
regiao_areiaArgilosa = [7,55,38]
regiao_areiaLamosa   = [23,55,25]
regiao_areiaSiltosa  = [38,55,15]


# ---- Função para gerar o nome do arquivo sem sobrescrever ----

def gerar_nome_arquivo_pasta(pasta):
    base = "diagrama_ternario"
    ext = ".png"
    caminho = os.path.join(pasta, base + ext)

    contador = 1
    while os.path.exists(caminho):
        caminho = os.path.join(pasta, f"{base}_{contador:02d}{ext}")
        contador += 1

    return caminho


# ---- Função principal: gerar e salvar o gráfico ----

def gerar_grafico():

    caminho = entry_arquivo.get().strip()
    titulo  = entry_titulo.get().strip()

    if caminho == "":
        messagebox.showerror("Erro", "Informe o caminho do arquivo CSV.")
        return

    if titulo == "":
        titulo = "Diagrama Ternário"

    try:
        dados = pd.read_csv(caminho, sep=",", index_col=0)

    except Exception as e:
        messagebox.showerror("Erro ao abrir arquivo", str(e))
        return

    # Criar dicionário de amostras
    samples1 = {
        f"{idx}": [row["Silte"], row["Areia"], row["Argila"]]
        for idx, row in dados.iterrows()
    }

    # ----- INICIA O GRÁFICO -----
    scale = 100
    figure, tax = ternary.figure(scale=scale)

    tax.set_title(titulo, fontsize=12, pad=20, ha='center', va='center')

    tax.left_axis_label("Argila (%)", fontsize=12, offset=0.14)
    tax.right_axis_label("Areia (%)", fontsize=12, offset=0.14)
    tax.bottom_axis_label("Silte (%)", fontsize=12, offset=0.14)

    # Linhas das classes texturais
    tax.plot(rot_list(linha_const(10)), color="black", linestyle="--", zorder=1)
    tax.plot(rot_list(linha_const(50)), color="black", linestyle="--", zorder=1)
    tax.plot(rot_list(linha_const(90)), color="black", linestyle="--", zorder=1)
    tax.plot(lista1, color="black", linestyle="--", zorder=1)
    tax.plot(lista2, color="black", linestyle="--", zorder=1)

    tax.boundary(False)

    # Regiões
    tax.annotate("Areia",  regiao_areias, ha="center", va="center", fontsize=8)
    tax.annotate("Silte",  regiao_siltes, ha="center", va="center", fontsize=8)
    tax.annotate("Argila", regiao_argilas,ha="center", va="center", fontsize=8)
    tax.annotate("Lama", regiao_lama, ha="center", va="center", fontsize=8)
    tax.annotate("Argila Arenosa", regiao_argilaArenosa, ha="center", va="center", fontsize=8)
    tax.annotate("Lama Arenosa", regiao_lamaArenosa, ha="center", va="center", fontsize=8)
    tax.annotate("Silte Arenoso", regiao_silteArenoso, ha="center", va="center", fontsize=8)
    tax.annotate("Areia\nArgilosa", regiao_areiaArgilosa, ha="center", va="center", fontsize=8)
    tax.annotate("Areia\nLamosa", regiao_areiaLamosa, ha="center", va="center", fontsize=8)
    tax.annotate("Areia\nSiltosa", regiao_areiaSiltosa, ha="center", va="center", fontsize=8)

    # Gradiente de cores
    N = len(samples1)
    colors = plt.cm.turbo(np.linspace(0, 1, N))

    # Plotar amostras
    for (label, point), c in zip(samples1.items(), colors):
        tax.scatter([point], marker="+", color=c, label=label, s=55, zorder=20)

    # Legenda fora do triângulo
    tax.legend(
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
        fontsize=8,
        frameon=True
    )

    tax.ticks(axis="l", multiple=10, linewidth=1, offset=0.02)
    tax.ticks(axis="r", multiple=10, linewidth=1, offset=0.02)
    tax.ticks(axis="b", multiple=10, linewidth=1, offset=0.02)

    tax.clear_matplotlib_ticks()

    for spine in figure.gca().spines.values():
        spine.set_visible(False)

    # ----------- SALVAR FIGURA NA MESMA PASTA DO CSV -----------
    pasta = os.path.dirname(caminho)
    caminho_saida = gerar_nome_arquivo_pasta(pasta)

    plt.savefig(caminho_saida, dpi=300, bbox_inches="tight")
    plt.close()

    messagebox.showinfo("Sucesso", f"Figura salva em:\n{caminho_saida}")


# ---------------- INTERFACE GRÁFICA ----------------

janela = tk.Tk()
janela.title("Gerador de Diagrama Ternário – Folk (1954)")
janela.geometry("600x220")

# Entrada do caminho do arquivo
tk.Label(janela, text="Caminho do arquivo CSV:").pack(anchor="w", padx=10)
entry_arquivo = tk.Entry(janela, width=70)
entry_arquivo.pack(padx=10)

def selecionar_arquivo():
    arq = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv")])
    entry_arquivo.delete(0, "end")
    entry_arquivo.insert(0, arq)

tk.Button(janela, text="Selecionar arquivo", command=selecionar_arquivo).pack(pady=5)

# Entrada do título
tk.Label(janela, text="Título do gráfico:").pack(anchor="w", padx=10)
entry_titulo = tk.Entry(janela, width=70)
entry_titulo.pack(padx=10)

# === Rodapé com autoria ===
label_autor = tk.Label(janela, text="Elaborado por Marcelo D. L. Jordão",
                       font=("Arial", 9, "italic"), fg="gray")
label_autor.pack(side=tk.BOTTOM, pady=5)

# Botão que gera o gráfico
tk.Button(janela, text="Gerar Gráfico Ternário", font=("Arial", 12, "bold"),
          command=gerar_grafico).pack(pady=15)
# === Mantém a interface rodando ===

# === Mostrar o gráfico na tela ===
plt.show()

janela.mainloop()

C:\Users\dilel\anaconda3\Lib\site-packages\ternary\plotting.py:148: UserWarning: No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
  ax.scatter(xs, ys, vmin=vmin, vmax=vmax, **kwargs)
